<img src="https://github.com/nicholasmetherall/digital-earth-pacific-macblue-activities/blob/main/attachments/images/DE_Pacific_banner.JPG?raw=true" width="900"/>

Figure 1.1.a. Jupyter environment + Python notebooks

# Digital Earth Pacific Notebook 1 prepare postcard and load data to csv

The objective of this notebook is to prepare a geomad postcard for your AOI (masking, scaling and loading additional band ratios and spectral indices) and sampling all the datasets into a csv based on your training data geodataframe.

## Step 1.1: Configure the environment

In [6]:
from datetime import datetime
import geopandas as gpd
import numpy as np
import pandas as pd
import joblib
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

In [7]:
# Predefined variable for title and version

# Enter your initials
initials = "nm"

# Date
date = datetime.now()

# Make a clean version string
version = f"{initials}-{date.strftime('%d%m%Y')}"
print(version)

nm-25072025


### Postcard csv

The objective of this notebook was to train the machine learning model that will allow us to classify an area with land cover classes defined through the training data.

Step 1.2. Input the training data to sample geomad data from the postcard

In [8]:
joined_df = gpd.read_file(f"training-data/{version}-joined_tdata.csv")
joined_df = joined_df.astype("float32")
joined_df

,cc_id,nir,red,blue,green,emad,smad,bcmad,nir08,nir09,...,mci,ndci,ln_bg,contrast,homogeneity,energy,ASM,correlation,mean,entropy
0,4.0,0.0358,0.0827,0.0897,0.1096,0.088441,2.352565e-06,0.000025,0.0312,0.0216,...,0.415796,0.020142,-0.200367,0.333333,0.866667,0.556250,0.309414,0.462352,12.402778,2.027906
1,4.0,0.0397,0.0899,0.0936,0.1165,0.093229,2.533943e-06,0.000021,0.0331,0.0235,...,0.427341,0.016411,-0.218861,0.208333,0.895833,0.641225,0.411169,0.537572,12.270833,1.701341
2,4.0,0.0418,0.0878,0.0926,0.1149,0.099894,2.388823e-06,0.000023,0.0373,0.0225,...,0.467039,0.009588,-0.215773,0.500000,0.800000,0.447149,0.199942,0.635032,12.152778,2.803573
3,4.0,0.0426,0.0888,0.0929,0.1147,0.101201,2.489907e-06,0.000024,0.0353,0.0246,...,0.462039,0.018785,-0.210796,0.208333,0.895833,0.580931,0.337481,0.652454,12.423611,1.946739
4,4.0,0.0414,0.0886,0.0918,0.1154,0.095467,2.378076e-06,0.000023,0.0345,0.0245,...,0.442781,0.026908,-0.228792,0.194444,0.902778,0.618017,0.381944,0.578595,12.361111,1.640143
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7431,2.0,0.2665,0.2228,0.1521,0.2058,0.140149,4.033267e-07,0.000008,0.2980,0.1895,...,1.130195,0.028347,-0.302367,10.222222,0.475031,0.218502,0.047743,0.694676,5.194445,5.222853
7432,2.0,0.3160,0.2412,0.1654,0.2212,0.119316,4.321992e-07,0.000007,0.3433,0.1915,...,1.410085,-0.036750,-0.290700,9.708333,0.505053,0.245523,0.060282,0.690502,4.868055,5.127813
7433,2.0,0.1952,0.1738,0.1284,0.1749,0.207139,1.295990e-06,0.000016,0.1927,0.1922,...,0.994903,0.060541,-0.309064,10.777778,0.429154,0.195187,0.038098,0.655073,5.625000,5.529737
7434,2.0,0.3069,0.2397,0.1637,0.2170,0.154166,4.589081e-07,0.000010,0.2626,0.1917,...,1.359770,-0.030082,-0.281862,8.777778,0.455440,0.216284,0.046779,0.662628,5.319445,5.282510


In [10]:
# joined_df=joined_df.drop(columns=["y", "x"])

In [11]:
print(len(joined_df.columns))
joined_df.columns

35


Index(['cc_id', 'nir', 'red', 'blue', 'green', 'emad', 'smad', 'bcmad',
       'nir08', 'nir09', 'swir16', 'swir22', 'coastal', 'rededge1', 'rededge2',
       'rededge3', 'mndwi', 'ndti', 'cai', 'ndvi', 'evi', 'savi', 'ndwi',
       'b_g', 'b_r', 'mci', 'ndci', 'ln_bg', 'contrast', 'homogeneity',
       'energy', 'ASM', 'correlation', 'mean', 'entropy'],
      dtype='object')

In [12]:
# The classes are the first column
classes = np.array(joined_df)[:, 0]

# The observation data is everything after the second column
observations = np.array(joined_df)[:, 1:]

# Create a model...
classifier = RandomForestClassifier()

# ...and fit it to the data
model = classifier.fit(observations, classes)

In [13]:
# Dynamically create the filename with f-string
file_path = f"models/{version}-test.model"

# Save the model
joblib.dump(model, file_path)

['models/nm-25072025-test.model']